### Import

In [2]:
import numpy as np
import pandas as pd
from pymongo import MongoClient

### Connect to Database

In [3]:
client = MongoClient()

In [4]:
client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

### Prepare Data

Read dataset, check the data types and the first 5 rows

In [5]:
df = pd.read_csv('../Data/insurance_data.csv', sep = ';')

In [6]:
df.dtypes

insuree#                      int64
gender                       object
is45OrOlder                 float64
isMarried                    object
hasKids                      object
insuredMonths                 int64
termLifeInsurance            object
multipleTermLifePolicies     object
healthInsurance              object
healthRiders                 object
premiumFrequency            float64
eStatements                  object
monthlyPremium               object
totalPremium                 object
renewal                      object
dtype: object

In [7]:
df.head(5)

,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal
0,1,F,0.0,Yes,Yes,23,Yes,No,No,NaN,12.0,Yes,"19,65","451,55",Y
1,2,F,1.0,No,No,42,Yes,Yes,Class A,3,1.0,Yes,"84,65","3541,35",N
2,3,F,0.0,Yes,No,72,Yes,No,No,NaN,12.0,No,"19,4","1496,45",Y
3,4,F,0.0,Yes,Yes,13,Yes,No,No,NaN,12.0,No,"19,55","265,3",Y
4,5,F,0.0,No,No,37,Yes,Yes,Class A,"3,4",1.0,No,"100,3","3541,4",Y


Clean dataset

In [8]:
df['is45OrOlder'] = df['is45OrOlder'].map({0: False, 1: True})
df['isMarried'] = df['isMarried'].map({'No': False, 'Yes': True})
df['hasKids'] = df['hasKids'].map({'No': False, 'Yes': True})
df['termLifeInsurance'] = df['termLifeInsurance'].map({'No': False, 'Yes': True})
df['multipleTermLifePolicies'] = df['multipleTermLifePolicies'].map({'No': False, 'Yes': True, 'no term life policy taken': False, 'term life policy not taken': False})
df['healthInsurance'] = df['healthInsurance'].map({'No': False, 'Class A': True, 'Class B': True})
df['healthRiders'] = df['healthRiders'].apply(lambda x : [int(i) for i in x.split(',')] if not pd.isna(x) else [])
df['eStatements'] = df['eStatements'].map({'No': False, 'Yes': True})
df['monthlyPremium'] = df['monthlyPremium'].str.replace(',', '.').astype(float)
df['totalPremium'] = df['totalPremium'].str.replace(',', '.').str.strip().apply(lambda x: float(x) if len(x) != 0 else np.nan)
df['renewal'] = df['renewal'].map({'N': False, 'Y': True})

Check data types and first 5 rows

In [9]:
df.dtypes

insuree#                      int64
gender                       object
is45OrOlder                  object
isMarried                      bool
hasKids                        bool
insuredMonths                 int64
termLifeInsurance              bool
multipleTermLifePolicies       bool
healthInsurance                bool
healthRiders                 object
premiumFrequency            float64
eStatements                    bool
monthlyPremium              float64
totalPremium                float64
renewal                        bool
dtype: object

In [10]:
df.head(5)

,insuree#,gender,is45OrOlder,isMarried,hasKids,insuredMonths,termLifeInsurance,multipleTermLifePolicies,healthInsurance,healthRiders,premiumFrequency,eStatements,monthlyPremium,totalPremium,renewal
0,1,F,False,True,True,23,True,False,False,[],12.0,True,19.65,451.55,True
1,2,F,True,False,False,42,True,True,True,[3],1.0,True,84.65,3541.35,False
2,3,F,False,True,False,72,True,False,False,[],12.0,False,19.40,1496.45,True
3,4,F,False,True,True,13,True,False,False,[],12.0,False,19.55,265.30,True
4,5,F,False,False,False,37,True,True,True,"[3, 4]",1.0,False,100.30,3541.40,True


Prepare data as a list of dictionaries for insertion to MongoDB

In [12]:
lst_dct_insertion = []
columns = ['insuree#', 'gender', 'is45OrOlder', 'isMarried', 'hasKids',
            'insuredMonths', 'eStatements', 'monthlyPremium', 'totalPremium', 'renewal']

for _, row in df.iterrows():
    doc_insertion = {}
    doc_insertion['_id'] = row['insuree#']

    for c in columns:
        doc_insertion[c] = row[c]

    doc_insertion['termLifeInsurance'] = {
        'hasPolicy': row['termLifeInsurance'],
        'hasMultiplePolicies': row['multipleTermLifePolicies']
    }

    doc_insertion['healthInsurance'] = {
        'hasPolicy': row['healthInsurance'],
        'riders': row['healthRiders']
    }

    premium_frequency = row['premiumFrequency']
    if not pd.isna(premium_frequency):
        premium_frequency = int(premium_frequency)
    doc_insertion['premiumFrequency'] = premium_frequency

    lst_dct_insertion.append(doc_insertion)

### Insert to Database

Check first document to be inserted

In [13]:
lst_dct_insertion[0]

{'_id': 1,
 'insuree#': 1,
 'gender': 'F',
 'is45OrOlder': False,
 'isMarried': True,
 'hasKids': True,
 'insuredMonths': 23,
 'eStatements': True,
 'monthlyPremium': 19.65,
 'totalPremium': 451.55,
 'renewal': True,
 'termLifeInsurance': {'hasPolicy': True, 'hasMultiplePolicies': False},
 'healthInsurance': {'hasPolicy': False, 'riders': []},
 'premiumFrequency': 12}

In [14]:
db = client.insurance
renewal = db.renewal

Delete any documents in collection

In [16]:
delete_res = renewal.delete_many({})
print(delete_res.deleted_count, "documents deleted.")

0 documents deleted.


Insert to collection

In [18]:
result = renewal.insert_many(lst_dct_insertion)
print(f"Number of documents inserted: {len(result.inserted_ids)}")

Number of documents inserted: 5500


Check first document inserted

In [19]:
renewal.find_one()

{'_id': 1,
 'insuree#': 1,
 'gender': 'F',
 'is45OrOlder': False,
 'isMarried': True,
 'hasKids': True,
 'insuredMonths': 23,
 'eStatements': True,
 'monthlyPremium': 19.65,
 'totalPremium': 451.55,
 'renewal': True,
 'termLifeInsurance': {'hasPolicy': True, 'hasMultiplePolicies': False},
 'healthInsurance': {'hasPolicy': False, 'riders': []},
 'premiumFrequency': 12}

### Close Connection to Database

In [20]:
client.close()